In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gzip
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

In [2]:
def images_file_read(file_name):
    with gzip.open(file_name, 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of images
        image_count = int.from_bytes(f.read(4), 'big')
        # third 4 bytes is the row count
        row_count = int.from_bytes(f.read(4), 'big')
        # fourth 4 bytes is the column count
        column_count = int.from_bytes(f.read(4), 'big')
        # rest is the image pixel data, each pixel is stored as an unsigned byte
        # pixel values are 0 to 255
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8).reshape((image_count, row_count, column_count))
        return images

In [3]:
def labels_file_read(file_name):
    with gzip.open(file_name, 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of labels
        label_count = int.from_bytes(f.read(4), 'big')
        # rest is the label data, each label is stored as unsigned byte
        # label values are 0 to 9
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

In [4]:
train_x = images_file_read('Dataset/mnist/train-images-idx3-ubyte.gz')
print(train_x.shape)

(60000, 28, 28)


In [5]:
train_x = np.reshape(train_x, (60000,784))
print(train_x.shape)

(60000, 784)


In [6]:
train_y = labels_file_read('Dataset/mnist/train-labels-idx1-ubyte.gz')
print(train_y.shape)

(60000,)


In [7]:
test_x = images_file_read('Dataset/mnist/t10k-images-idx3-ubyte.gz')
print(test_x.shape)

(10000, 28, 28)


In [8]:
test_x = np.reshape(test_x, (10000,784))
print(train_y.shape)

(60000,)


In [9]:
test_y = labels_file_read('Dataset/mnist/t10k-labels-idx1-ubyte.gz')
print(test_y.shape)

(10000,)


In [11]:
pca = PCA(n_components=15, random_state=0)
train_x = pca.fit_transform(train_x)
test_x = pca.transform(test_x)

In [15]:
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(60000, 15) (60000,)
(10000, 15) (10000,)


In [18]:
classes = [0,1,2,3,4,5,6,7,8,9]

In [21]:
class_wise_data = {}
for c in range(len(classes)):
    class_wise_data[c] = []

In [22]:
unique, counts = np.unique(train_y, return_counts=True)
print(np.asarray((unique, counts)).T)

[[   0 5923]
 [   1 6742]
 [   2 5958]
 [   3 6131]
 [   4 5842]
 [   5 5421]
 [   6 5918]
 [   7 6265]
 [   8 5851]
 [   9 5949]]


In [26]:
for itr in range(train_y.shape[0]):
    class_wise_data[train_y[itr]].append(train_x[itr])

In [32]:
for c in range(len(classes)):
    class_wise_data[c] = np.asarray(class_wise_data[c])

In [36]:
class_wise_mean = {}
for c in range(len(classes)):
    class_wise_mean[c] = np.mean(class_wise_data[c], axis=0)

In [37]:
overall_mean = np.mean(train_x, axis=0)

In [38]:
overall_mean.shape

(15,)

In [39]:
class_wise_mean[0].shape

(15,)

In [40]:
class_wise_data[0].shape

(5923, 15)

In [42]:
Sk_list = []
for class_c, mean_c in class_wise_mean.items():
    sub = np.subtract(class_wise_data[class_c], mean_c)
    Sk_list.append(np.dot(np.transpose(sub), sub))

In [43]:
Sk_array = np.asarray(Sk_list)

In [44]:
Sk_array.shape

(10, 15, 15)

In [45]:
Sw = np.sum(Sk_array, axis=0)

In [46]:
Sw.shape

(15, 15)

In [47]:
Nk = {}
for class_c in range(len(classes)):
    Nk[class_c] = class_wise_data[class_c].shape[0]

In [48]:
Sb = []
for class_c, mean_c in class_wise_mean.items():
    sub2 = mean_c - overall_mean
    Sb.append(np.multiply(Nk[class_c], np.outer(sub2, sub2.T)))
Sb = np.sum(Sb, axis=0)

In [49]:
Sb.shape

(15, 15)

In [50]:
u,s,v = np.linalg.svd(Sw, full_matrices=True)
Swinv = np.dot(v.transpose(),np.dot(np.diag(s**-1),u.transpose()))
mat = np.dot(Swinv,Sb)
eigen_values, eigen_vectors = np.linalg.eigh(mat)
eiglist = [(eigen_values[i], eigen_vectors[:, i]) for i in range(len(eigen_values))]
eiglist = sorted(eiglist, key=lambda x: x[0], reverse=True)
W = np.array([eiglist[i][1] for i in range(len(classes)-1)])
W = np.asarray(W).T

In [51]:
train_x_projected = np.dot(train_x, W)
test_x_projected = np.dot(test_x, W)

In [52]:
lda = LinearDiscriminantAnalysis()
lda.fit(train_x_projected,train_y)

LinearDiscriminantAnalysis()

In [53]:
pred = lda.predict(test_x_projected)

In [54]:
print('Overall accuracy : ',lda.score(test_x_projected,test_y)*100, '%')

Overall accuracy :  79.61 %


In [55]:
c_matrix = confusion_matrix(test_y, pred)
class_wise_accuracy = 100*c_matrix.diagonal()/c_matrix.sum(axis=1)

In [56]:
for y in range(len(class_wise_accuracy)):
    print('Class wise accuracy of class ',y,' : ',class_wise_accuracy[y], '%')

Class wise accuracy of class  0  :  89.38775510204081 %
Class wise accuracy of class  1  :  93.12775330396475 %
Class wise accuracy of class  2  :  75.09689922480621 %
Class wise accuracy of class  3  :  80.0990099009901 %
Class wise accuracy of class  4  :  83.09572301425662 %
Class wise accuracy of class  5  :  58.63228699551569 %
Class wise accuracy of class  6  :  82.04592901878914 %
Class wise accuracy of class  7  :  83.36575875486382 %
Class wise accuracy of class  8  :  71.14989733059548 %
Class wise accuracy of class  9  :  76.21407333994054 %
